## SRJ - Verification of H - H4 case

In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'H 0 0 0; H 0 0 3.0; H 0 0 6.0; H 0 0 9.0 ',
    basis = 'sto-6g',
    spin = 0)
rhf = scf.RHF(mol_pyscf)
rhf.kernel()

import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)

#Changed core and active orbitals
core=[] 
active=[0,1,2,3]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)

converged SCF energy = -1.33256885786978
The value of core: [0.76436708]
Total orbitals are 4
(-1.1078408272269964+0j) * I(0) + (0.05063640903526401+0j) * Z(0) + (0.002022956774996024+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (0.002022956774996024+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + (0.04498894667812685+0j) * Z(2) + (0.025791740314689483+0j) * (Z(0) @ Z(2)) + (0.002118173348982716+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (0.002118173348982716+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (-0.006238714417889932+0j) * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (-0.006238714417889932+0j) * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (0.03841020409884824+0j) * Z(4) + (0.032993771640804546+0j) * (Z(0) @ Z(4)) + (0.032312630567687045+0j) * Z(6) + (0.03866341961135375+0j) * (Z(0) @ Z(6)) + (0.05063640903526405+0j) * Z(1) + (0.07316429942261345+0j) * (Z(0) @ Z(1)) + (0.008899910565698449+0j) * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + (0.008899910565698449+0j) * (X(0) @ Z(2) @ Z(3) @ X(4)) + (0.04

In [2]:
qubits = 8
active_electrons = 4
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("lightning.qubit", wires = qubits)
@qml.qnode(dev)
def e():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

fe = e()
print('fe', fe)

fe -1.3325688578697834


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


In [ ]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 8
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1.3325688578697834
Total excitations are 26
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.3100451660355517 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [1.12299277]
Final cost: -1.4300836524422342
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The adapt iteration now is 1
The highest operator value is 0.3633341837331607 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]


...

The highest operator value is 8.257013248275102e-07 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [ 1.19805075e+00  9.12115452e-01  1.30019627e+00  1.20013695e+00
  1.64476452e+00  1.62460789e+00 -1.54777143e+00 -1.29589832e+00
 -1.56475866e+00 -1.29548440e+00  8.70161888e-01  9.28906825e-01
 -1.45302783e+00 -1.45380096e+00  4.90893327e+00  3.15141333e+00
 -7.24537211e-01 -4.84454226e-03 -1.69109418e+00  3.23752037e-06
  1.51556179e-04 -1.09530025e-04  3.35442320e-05 -6.91811548e-06]
Final cost: -1.8851248993392882
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.91) ···
1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.91) ···
2: ──X─╭FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(0.91) ···
3: ──X─├FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(0.91) ···
4: ────│────────────────────────────────│─────────────────────────────── ···
5: ────│───────────────────────────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 6.960353720331103e-07 for operator a⁺(0) a⁺(1) a(6) a(7)
Highest gradient excitation is [0, 1, 6, 7]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [ 1.19805078e+00  9.12113040e-01  1.30019600e+00  1.20013312e+00
  1.64476464e+00  1.62460700e+00 -1.54777079e+00 -1.29590216e+00
 -1.56475894e+00 -1.29548563e+00  8.70163325e-01  9.28906350e-01
 -1.45302900e+00 -1.45380251e+00  4.90893301e+00  3.15141236e+00
 -7.24537822e-01 -4.84465712e-03 -1.69109417e+00  1.68219887e-06
  1.53071598e-04 -1.11480818e-04  3.43462122e-05 -7.54055125e-06
  2.82768330e-06]
Final cost: -1.8851248993416203
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.91) ···
1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.91) ···
2: ──X─╭FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(0.91) ···
3: ──X─├FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(0.91) ···
4: ────│────────────────────────────────│─────────────────────────────── ···
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.91) ···
6: ────├FermionicDoubleExcitation(1.20)─╰FermionicDoubleExcitation(0.91)

In [4]:
FCI = -1.88512490
energies[-1] - FCI

6.583797951265069e-10

## To find out core and active orbitals

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value


        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization



        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)


        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H

In [ ]:
symbols  = [ 'H', 'H', 'H', 'H']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0*r_bohr, 0.0*r_bohr], [0.0, 0.0, 3.0*r_bohr], [0.0,0, 6.0*r_bohr], [0, 0, 9.0*r_bohr]])

print(coordinates)

electrons = 4  # 7 from N and 3 from H
orbitals = 8 #Spin orbitals  
charge = 0

active_electrons = 4 #No of electrons
active_orbitals = 4 #Spatial orbitals


gs_state, params, ash_excitation, qubits, H = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1)

H2O-0.6A-GS+ES-DBS
[[ 0.          0.          0.        ]
 [ 0.          0.          5.66917797]
 [ 0.          0.         11.33835593]
 [ 0.          0.         17.0075339 ]]
Using active space, check if you change the H accordingly
core orbitals: []
active orbitals: [0, 1, 2, 3]
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Current parameters: [ 1.12299270e+00 -2.59511114e-09]
Current cost: -1.4300836708403268

Current parameters: [ 1.12299270e+00 -2.59511114e-09]
Current cost: -1.4300836708403275

Final updated parameters: [ 1.12299270e+00 -2.59511114e-09]
Final cost: -1.4300836708403275
